In [12]:
%pip install keplergl==0.3.2 --quiet

StatementMeta(, 6d40f30a-3b68-47f1-913e-6f896a8fb24d, -1, Finished, Available)


[notice] A new release of pip is available: 23.0 -> 23.2
[notice] To update, run: /nfs4/pyenv-ab4e68c9-d6b0-4e48-8a91-b21cc4fed27d/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from keplergl import KeplerGl
import pyspark.sql.functions as F
import h3

def create_kepler_html(data, config, height):
  map_1 = KeplerGl(height=height, data=data, config=config)
  html = map_1._repr_html_().decode("utf-8")
  new_html = html + """<script>
      var targetHeight = "{height}px";
      var interval = window.setInterval(function() {{
        if (document.body && document.body.style && document.body.style.height !== targetHeight) {{
          document.body.style.height = targetHeight;
        }}
      }}, 250);</script>""".format(height=height)
  return new_html

StatementMeta(, 6d40f30a-3b68-47f1-913e-6f896a8fb24d, 26, Finished, Available)

In [15]:
import pyspark.sql.functions as F
def load_stats_from_commit(commit_file):
  df = spark.read.json(commit_file).where("add is not null")
  add_schema = """
  struct
  <
    numRecords:long,
    minValues: struct<startLat: double,startLon: double>,
    maxValues: struct<startLat: double,startLon: double>
  >
  """

  stats = (
    df
      .select("add.path", "add.size", F.from_json("add.stats", add_schema).alias("stats"), F.col("add.stats").alias("copy"))
      .selectExpr(
        "size", "stats.minValues.startLat as min_pickup_lat","stats.minValues.startLon as min_pickup_lon", 
        "stats.maxValues.startLat as max_pickup_lat","stats.maxValues.startLon as max_pickup_lon"
      )
  )

  stats = (
    stats
      .withColumn("geometry", F.expr(
        """
          concat('POLYGON ((' , 
            min_pickup_lon, ' ', min_pickup_lat, ',' ,
            max_pickup_lon, ' ', min_pickup_lat, ',' ,
            max_pickup_lon, ' ', max_pickup_lat, ',' ,
            min_pickup_lon, ' ', max_pickup_lat, ',' ,
            min_pickup_lon, ' ', min_pickup_lat, 
          '))'
          )
      """))
  )

  return stats

StatementMeta(, 6d40f30a-3b68-47f1-913e-6f896a8fb24d, 27, Finished, Available)

In [16]:
stats = load_stats_from_commit("Files/nyc-yellow-taxi/plain-trips/_delta_log/00000000000000000000.json")
display(stats)

StatementMeta(, 6d40f30a-3b68-47f1-913e-6f896a8fb24d, 28, Finished, Available)

SynapseWidget(Synapse.DataFrame, dd79113e-1f59-45a9-87a7-f08ba6b12700)

StatementMeta(, 6d40f30a-3b68-47f1-913e-6f896a8fb24d, 13, Finished, Available)

In [26]:
map_config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
      ],
      "layerBlending": "subtractive",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": False,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 40.79223909866621,
      "longitude": -73.99527316762153,
      "pitch": 0,
      "zoom": 8,
      "isSplit": False
    },
  }
}


StatementMeta(, 6d40f30a-3b68-47f1-913e-6f896a8fb24d, 38, Finished, Available)

In [27]:
plain_table_html = create_kepler_html(
    data= {"data_frame": stats.toPandas() }, 
    config=map_config, 
    height=600
)
 
displayHTML(plain_table_html)

StatementMeta(, 6d40f30a-3b68-47f1-913e-6f896a8fb24d, 39, Finished, Available)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [28]:
optimised_table_stats = load_stats_from_commit("Files/nyc-yellow-taxi/optimised-trips/_delta_log/00000000000000000001.json")
display(optimised_table_stats)

StatementMeta(, 6d40f30a-3b68-47f1-913e-6f896a8fb24d, 40, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7b9c8b55-fcca-4c23-bfd7-4076134b7995)

In [29]:
optimised_table_html = create_kepler_html(
    data= {"data_frame": optimised_table_stats.toPandas() }, 
    config=map_config, 
    height=600
)
displayHTML(optimised_table_html)

StatementMeta(, 6d40f30a-3b68-47f1-913e-6f896a8fb24d, 41, Finished, Available)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


### Needle in haystack query

In [31]:
%%sql

SELECT * FROM delta.`Files/nyc-yellow-taxi/plain-trips`
WHERE startLat = 40.756122 AND startLon = -73.989682

StatementMeta(, 6d40f30a-3b68-47f1-913e-6f896a8fb24d, 43, Finished, Available)

<Spark SQL result set with 1 rows and 23 fields>

- Duration is 4 minutes
- Number of files read 64 (which is all parquet files in non-optimised table)
- Size of files read 38 GB
- Number of records in such files is ~ 1.2 B

![full-scan](https://i.imgur.com/uZJup3q.png)

In [32]:
%%sql

SELECT * FROM delta.`Files/nyc-yellow-taxi/optimised-trips`
WHERE startLat = 40.756122 AND startLon = -73.989682

StatementMeta(, 6d40f30a-3b68-47f1-913e-6f896a8fb24d, 44, Finished, Available)

<Spark SQL result set with 1 rows and 23 fields>

- Duration is 26 seconds
- Number of files read 3 (which is a subset of all parquet files in table)
- Size of files read 3 GB
- Number of records in such files is ~ 63 M

![optimised-scan](https://i.imgur.com/pPjr4fk.png)

<hr>

Some Spark environment display more metrics like number of files pruned.

![pruned-files-info](https://i.imgur.com/WGep0Nq.png)


See [this article](https://yousry.medium.com/delta-lake-z-ordering-from-a-to-z-315063a42031) for details on Z-Order.

